In [ ]:
import pandas as pd

pacientes = pd.read_csv(r'../Data/HC_Janeiro2021/HC_PACIENTES_1.csv', sep='|')

df = pd.read_csv(r'../Data/HC_Janeiro2021/HC_EXAMES_1.csv', sep='|')
df.insert(1, "EXAME", df['DE_EXAME']+" "+df['DE_ANALITO'])

out = df.pivot_table(index='ID_aTENDIMENTO', columns='EXAME', values='DE_RESULTADO', aggfunc='first')

In [51]:
import numpy as np

valuesTable = []

for column in out.columns:
    uniqueValues = out[column].unique()
    notNullVal = uniqueValues[~pd.isnull(uniqueValues)]
    values = notNullVal.transpose()

    valuesTable.append(values)

stringValues = []

for values in valuesTable:
    for value in values:
        try:
            float(value)
        except:
            stringValues.append(value)

stringValues = pd.DataFrame(stringValues)[0].unique()
print(stringValues)

['< 4.0' 'Indetectavel' '*' ... '>20.0' '< 2.40' '<2.40']


In [ ]:
def has_digits(line):
    return any(char.isdigit() for char in line)

clearStringVal = []

for val in stringValues:
    if not has_digits(val):
        clearStringVal.append(val)

equivalenceMatrix = []
negativeCases = ["neg", "*", "não", "nao", "raro", "rara", "isolados", "inadequada", "zero", 
                 "inválido", "invalido", "ausência", "ausencia", "failed", "indetectavel", "-",
                 "mínima", "minima", "discreta"]
singleCases = ["*", "r", "s", "i", "n", "w", "-"]

for val in clearStringVal:
    if val.count != 1 and any(substring in val.lower() for substring in negativeCases):
        equivalenceMatrix.append((val, 0))
    elif val.count == 1 and any(substring in val.lower() for substring in singleCases):
        equivalenceMatrix.append((val, 0))
    else:
        equivalenceMatrix.append((val, 1))

print(equivalenceMatrix)

[('Indetectavel', 0), ('*', 0), ('Negativa', 0), ('Reação citoquímica realizada para complementar diagnóstico.', 1), ('Foi  realizada a reação citoquímica.', 1), ('Presente', 1), ('Imediata', 1), ('NEGATIVO', 0), ('Realizado', 1), ('S', 1), ('R', 1), ('Não reagente', 0), ('Reagente', 1), ('negativo', 0), ('Positivo', 1), ('Negativo', 0), ('Duvidoso', 1), ('Não Reagente', 0), ('Ácido úrico', 1), ('.', 1), ('presente', 1), ('ausente', 1), ('Ausente', 1), ('Teste realizado através de curva de calibração híbrida para HBPM e HFN.', 1), ('Não detectado', 0), ('Detectado', 1), ('Inconclusivo', 1), ('Cocos Gram Positivos', 1), ('Não foram observados microrganismos', 0), ('Bacilos Gram Negativos', 0), ('Cocos Gram Positivos aos Pares e em Cadeias', 1), ('Leveduras', 1), ('Cocos Gram Positivos aos Pares', 1), ('Cocos Gram Positivos Agrupados', 1), ('Bacilos Gram Positivos', 1), ('Diplococos Gram Negativos', 0), ('Límpido e incolor', 1), ('Límpido e xantocrômico', 1), ('Levemente  hemorrágico', 1